<a href="https://colab.research.google.com/github/irythmgarg/LINEAR_REGRESSION_TO_PREDICT_RATE_OF_A_PIZZA/blob/main/LINEAR_REGRESSION_TO_PREDICT_RATE_OF_A_PIZZA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [77]:
import pandas as pd
import numpy as np

In [78]:
df=pd.read_csv('/content/ridhampizza.csv');
df.head()

,id,date,time,name,size,type,price
0,2015-000001,1/1/2015,11:38:36,hawaiian,M,classic,13.25
1,2015-000002,1/1/2015,11:57:40,classic_dlx,M,classic,16.00
2,2015-000002,1/1/2015,11:57:40,mexicana,M,veggie,16.00
3,2015-000002,1/1/2015,11:57:40,thai_ckn,L,chicken,20.75
4,2015-000002,1/1/2015,11:57:40,five_cheese,L,veggie,18.50


In [79]:
df.drop(columns=['id',	'date',	'time'],inplace=True);
df.head()

,name,size,type,price
0,hawaiian,M,classic,13.25
1,classic_dlx,M,classic,16.00
2,mexicana,M,veggie,16.00
3,thai_ckn,L,chicken,20.75
4,five_cheese,L,veggie,18.50


In [80]:
x=df.iloc[:,:3]
y=df.iloc[:,-1]
x.head()

,name,size,type
0,hawaiian,M,classic
1,classic_dlx,M,classic
2,mexicana,M,veggie
3,thai_ckn,L,chicken
4,five_cheese,L,veggie


In [93]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.1)
x_train.sample(5)


,name,size,type
40,four_cheese,L,veggie
21,classic_dlx,S,classic
3,thai_ckn,L,chicken
47,spinach_fet,L,veggie
0,hawaiian,M,classic


In [95]:
x_train.shape

(45, 3)

In [96]:
x_test

,name,size,type
10,bbq_ckn,S,chicken
9,ital_supr,M,supreme
27,cali_ckn,M,chicken
46,four_cheese,M,veggie
39,thai_ckn,L,chicken


In [94]:
from sklearn.preprocessing import OneHotEncoder
oe=OneHotEncoder(sparse=False,drop='first');
x_train_ohe_name_type=oe.fit_transform(x_train[['name','type']]);
x_test_ohe_name_type=oe.transform(x_test[['name','type']]);

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [97]:
y=pd.DataFrame(x_train['size'],columns=['size'])
x_test['size']
z=pd.DataFrame(x_test['size'],columns=['size'])
z

,size
10,S
9,M
27,M
46,M
39,L


In [98]:
from sklearn.preprocessing import OrdinalEncoder
ord=OrdinalEncoder(categories=[['S','M','L']]);
x_train_ord_size=ord.fit_transform(y);
x_test_ord_size=ord.transform(z);

In [99]:
x_train_transformed=np.concatenate((x_train_ohe_name_type,x_train_ord_size),axis=1)
x_test_transformed=np.concatenate((x_test_ohe_name_type,x_test_ord_size),axis=1)

In [100]:
x_train_transformed.shape

(45, 28)

In [101]:
x_train_new=pd.DataFrame(x_train_transformed);
x_test_new=pd.DataFrame(x_test_transformed);

In [102]:
from sklearn.compose import ColumnTransformer
ct=ColumnTransformer(transformers=[
    ('tnf1',OneHotEncoder(sparse=False,drop='first'),['name','type']),
    ('tnf2',OrdinalEncoder(categories=[['S','M','L']]),['size'] )
],remainder='passthrough'
)

In [105]:
x_train_dfct=pd.DataFrame(ct.fit_transform(x_train))
x_test_dfct=pd.DataFrame(ct.transform(x_test))


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [114]:
from sklearn.linear_model import LinearRegression
lr=LinearRegression()
lr.fit(x_train_dfct,y_train)
y_ans=lr.predict(x_test_dfct)


In [115]:
y_test

10    12.75
9     16.50
27    16.75
46    14.75
39    20.75
Name: price, dtype: float64

In [116]:
from sklearn.metrics import r2_score

In [117]:
print(r2_score(y_test,y_ans));

0.9759738892233555
